# Banco de dados
Aqui nós criamos o banco de dados hm1. Ele conterá os dados coletados da página do autor desejado, utilizando a função scrape

In [1]:
from sqlite3 import dbapi2 as sq3
import os
import sqlite3
import pandas as pd
import altair as alt
import networkx as nx
import matplotlib.pyplot as plt
from scraper_scholar import * 


In [2]:
dados = scrape('Jeffrey Heer') #Coletando informações pela função scrape
print(dados[0:5])

C:\Users\julia\Google Drive\MESTRADO\EMAP-FGV\Fundamentos de Dada Scientists\hm1\scraper_scholar.py:44: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 44 of the file C:\Users\julia\Google Drive\MESTRADO\EMAP-FGV\Fundamentos de Dada Scientists\hm1\scraper_scholar.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  html = BeautifulSoup(driver.page_source)


[{'title': 'D3: Data-Driven Documents', 'authors': ['M Bostock', 'V Ogievetsky', 'J Heer']}, {'title': 'Prefuse: a toolkit for interactive information visualization', 'authors': ['J Heer', 'SK Card', 'JA Landay']}, {'title': 'Vizster: Visualizing online social networks', 'authors': ['J Heer', 'D Boyd']}, {'title': 'Narrative visualization: Telling stories with data', 'authors': ['E Segel', 'J Heer']}, {'title': 'Crowdsourcing graphical perception: using mechanical turk to assess visualization design', 'authors': ['J Heer', 'M Bostock']}]


Construindo o banco de dados

In [3]:
ourschema = '''
    DROP TABLE IF EXISTS "author";
    DROP TABLE IF EXISTS "paper";
    DROP TABLE IF EXISTS "author_paper";
    CREATE TABLE "author" (
        "id" INTEGER PRIMARY KEY  NOT NULL ,
        "author" VARCHAR NOT NULL UNIQUE
    );
    CREATE TABLE "paper" ( 
        "id" INTEGER PRIMARY KEY  NOT NULL ,
        "paper" VARCHAR NOT NULL UNIQUE
    );
    CREATE TABLE "author_paper" (
        'author_id' VARCHAR,
        'paper_id' VARCHAR,
        FOREIGN KEY(author_id) REFERENCES author(id)
        FOREIGN KEY(paper_id) REFERENCES paper(id)
    );
    '''

In [4]:
PATHSTART="."
def get_db(dbfile):
    sqlite_db = sq3.connect(os.path.join(PATHSTART, dbfile))
    return sqlite_db
def init_db(dbfile, schema):
    """Creates the database tables."""
    db = get_db(dbfile)
    db.cursor()
    db.executescript(schema)
    db.commit()
    return db

In [5]:
db=init_db("hw1.db", ourschema)

In [6]:
#criando data papers, authors e authors_paper
x = []
for i in dados:
    for j in i['authors']:
        x.append([i['title'],j.strip()])
x = pd.DataFrame(x,columns=['paper','author'])

x1 = list(sorted(set(x['paper'])))
x2 = list(sorted(set(x['author'])))
paper=pd.DataFrame(x1,columns=['paper'])

author=pd.DataFrame(x2,columns=['author'])    

In [8]:
paper.head(5)

,paper
0,A Visual Inspection of Online Health Communities
1,A demonstration of the BigDAWG polystore system
2,A taxonomy of tools that support the fluent an...
3,A tour through the visualization zoo.
4,Active capture: automatic direction for automa...


Inserindo dados

In [9]:
author.to_sql("author", db, if_exists="append", index=False)
paper.to_sql("paper", db, if_exists="append", index=False)


In [10]:
def make_query(tabela):
    c=db.cursor().execute("SELECT * FROM {};".format(tabela))
    return c.fetchall()

In [11]:
a = make_query('author')
t = make_query('paper')
a = pd.DataFrame(a,columns=['author_id','author'])
t = pd.DataFrame(t,columns=['paper_id','paper'])
author_paper = pd.merge(x,a,on='author') 
author_paper = pd.merge(author_paper,t,on='paper') 



In [12]:
author_paper.head(5)

,paper,author,author_id,paper_id
0,D3: Data-Driven Documents,M Bostock,163,39
1,D3: Data-Driven Documents,V Ogievetsky,261,39
2,D3: Data-Driven Documents,J Heer,123,39
3,Crowdsourcing graphical perception: using mech...,M Bostock,163,37
4,Crowdsourcing graphical perception: using mech...,J Heer,123,37


In [17]:
#author_paper = pd.concat([author_paper['author_id'],author_paper['paper_id']],axis=1)
author_paper.to_sql("author_paper", db, if_exists="append", index=False,)